In [54]:
#必要なライブラリのインポート
import pandas as pd
import sqlite3

In [55]:
#データベースファイルへの接続 / SQLコマンドの実行 / クローズ
conn = sqlite3.connect("hobby_data_cleaned.db")
df = pd.read_sql_query("SELECT * FROM hobbies", conn)
conn.close()

# ここで df_unique2 を再定義、dfはPython（メモリ上）のPandas DataFrameとなっている
df_unique2 = df.copy()

In [58]:
df_unique2

,カテゴリ,趣味名,趣味詳細説明文,hybrid_score
0,スポーツ,ライブ観戦,"ライブは""鑑賞する""、""聴く""というよりも、音楽を楽しみつつ、その場の臨場感も楽しむことから...",0
1,スポーツ,アーチェリー,みなさん、アーチェリーといえば、何を思い浮かべますか？\n2004年アテネオリンピックで、「...,0
2,スポーツ,水泳,水泳と言えば、子供に習わせたい習い事の上位にランクインされる人気のスポーツのひとつ。\nしか...,0
3,スポーツ,テニス,テニスというと最近では錦織選手を筆頭に、世界で活躍する選手が増えてきました。\nもとより人気...,0
4,スポーツ,乗馬,乗馬と聞くと、『お金持ちや貴族など、特別な人のためのスポーツ』そんな印象を持っていませんか？...,0
...,...,...,...,...
338,その他,ボランティア活動,みなさんはボランティア活動に興味がありますか？\nいま話題の東京オリンピックでは9万人という...,0
339,その他,小説執筆,本を読むのは好きだけど、自分で書くのは難しい・・・\nそう思ってあきらめたことはありませんか...,0
340,その他,手話,手話と言うと、まず思い浮かべるのは、ボランティアでしょうか？\n手話を知っていれば、耳が不自...,0
341,その他,エッセイ,誰もが一度は耳にしたことがある「エッセイ」。\nエッセイと聞くと、「堅苦しそう」「難しそう」...,0


In [59]:
######マッチングのための前処理##########
#まずpowershellでこちらを実行する　pip install sentence-transformers

#from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import os
os.environ["HF_HOME"] = "./hf_cache"  # 別フォルダにキャッシュを保存

In [60]:
# １．類義語リストを辞書形式
synonym_dict = {
    "体幹を鍛える": [
        "体幹", "バランス", "姿勢強化",
        "インナーマッスル", "軸が安定する", "安定感", "体の中心", "体幹トレーニング", "コア", "胴体強化", "体のブレが減る", "ケガ予防", "姿勢改善"
    ],
    "姿勢がよくなる": [
        "美姿勢", "姿勢改善", "背筋が伸びる",
        "猫背解消", "背筋", "自然な立ち姿", "自信がつく", "姿勢美人", "印象が良くなる", "健康的", "体が整う", "見た目が若返る", "骨盤矯正"
    ],
    "代謝が上がる": [
        "ダイエット", "脂肪燃焼", "体温上昇",
        "痩せる", "汗をかく", "基礎代謝", "内臓脂肪燃焼", "エネルギー消費", "太りにくくなる", "デトックス", "筋トレ効果", "血流促進", "引き締め"
    ],
    "リラックス": [
        "ストレス解消", "安らぐ", "落ち着く",
        "癒し", "深呼吸", "緊張がほぐれる", "穏やか", "気持ちが和らぐ", "安心感", "癒される時間", "副交感神経", "気持ちが整う", "気分転換"
    ],
    "癒される": [
        "癒し", "心が和む", "ほっこりする",
        "優しい時間", "あたたかい気持ち", "穏やかになる", "心が休まる", "精神的な回復", "心地よい", "優しい空気", "穏やかな風景", "満たされる", "癒し効果"
    ],
    "ワクワクする": [
        "興奮", "テンションが上がる", "ドキドキ",
        "冒険心", "期待感", "楽しい予感", "ハイテンション", "わくわく感", "童心にかえる", "躍動感", "高揚感", "やってみたい気持ち", "好奇心が刺激される"
    ],
    "集中する": [
        "没頭", "ゾーンに入る", "集中力が増す",
        "時間を忘れる", "夢中になる", "一点集中", "余計なことを忘れる", "作業効率アップ", "雑念が消える", "集中状態", "集中タイム", "深く入り込む", "生産性向上"
    ],
    "非日常感を味わう": [
        "刺激的", "冒険", "日常から離れる",
        "特別感", "旅行気分", "新しい体験", "旅に出る", "リゾート感", "新鮮さ", "異世界", "気分転換", "脱日常", "未知との出会い"
    ],
    "表現力が身につく": [
        "アウトプット力", "感情表現", "自己表現",
        "発信力", "伝える力", "創作活動", "話し方が上手くなる", "伝達能力", "感性が磨かれる", "自分を表す", "創造的思考", "魅せ方を学ぶ", "パフォーマンス力"
    ],
    "観察力が鍛える": [
        "洞察", "気づき", "注意深くなる",
        "観察眼", "細部に気づく", "よく見る力", "鋭い視点", "注意力", "客観視", "冷静な分析", "周囲をよく見る", "観察力アップ", "データに強くなる"
    ],
    "創造力を育てる": [
        "クリエイティブ", "想像力", "アイデア",
        "発想力", "ものづくり", "企画力", "アート思考", "柔軟な発想", "創作力", "イメージ力", "新しい視点", "ゼロから作る", "発明"
    ],
    "達成感を味わう": [
        "やりがい", "目標達成", "満足感",
        "チャレンジ成功", "自分に勝つ", "完遂感", "達成した気持ち", "報われる", "頑張った成果", "やった！と思える", "自信がつく", "満たされる", "積み重ねの成果"
    ],
    "文化に触れる": [
        "伝統文化", "アート", "知的好奇心",
        "歴史を学ぶ", "海外の風習", "芸術鑑賞", "クラシック", "古典", "民俗芸能", "日本文化", "美術館", "伝統芸能", "文化的教養"
    ],
    "自然と関わる": [
        "自然体験", "屋外", "アウトドア",
        "緑に触れる", "森林浴", "キャンプ", "自然観察", "四季を感じる", "田舎体験", "野外活動", "花や植物", "地球とつながる", "ナチュラル志向"
    ],
    "一人で楽しむ": [
        "ソロ", "マイペース", "自分の時間",
        "一人旅", "内省的", "集中できる", "人に気を使わない", "自分軸", "孤独を楽しむ", "ひとり趣味", "自由時間", "自己充電", "気まま"
    ],
    "仲間と楽しむ": [
        "グループ", "交流", "コミュニティ",
        "チーム活動", "一緒に笑う", "つながり", "共同作業", "友達と楽しむ", "人との関係", "対話", "社交性", "仲間意識", "連帯感"
    ],
    "初期費用が安い": [
        "低コスト", "安価", "コスパがいい",
        "節約できる", "お金がかからない", "無料ではじめられる", "格安", "お手頃価格", "続けやすい", "経済的", "自己投資が少ない", "出費が少ない", "維持費が安い"
    ],
    "自宅でできる": [
        "家でできる", "在宅", "家時間",
        "家から出なくていい", "室内", "雨でもできる", "防音不要", "家の中", "オンライン", "ホームエクササイズ", "自宅趣味", "省スペース", "リモート対応"
    ],
    "気軽に始められる": [
        "初心者OK", "始めやすい", "敷居が低い",
        "すぐにできる", "準備が簡単", "誰でもできる", "ハードルが低い", "初期知識不要", "導入が楽", "とっつきやすい", "カジュアル", "気負わずできる", "始めやすさ"
    ]
}

In [61]:
user_input = "あなたは趣味を通じて姿勢がよくなる、集中する、表現力を味わうことを望んでいます。またそれは一人で楽しむことができ、なおかつ自宅でできるものであると、より満足感が高まります。"

In [71]:
# ユーザーの回答に含まれるキーワード
user_keywords = ["体幹を鍛える", "姿勢がよくなる", "代謝が上がる"]

In [ ]:
# マッチングのルールを作成（ユーザーが回答したキーワードをカバーしている数が多いほどポイントが高い、更にすべてカバーしている場合にマッチしたキーワードの数が多ければボーナスポイント追加）
def hybrid_keyword_score(user_input, user_keywords, synonym_dict):
    covered_count = 0
    total_match_count = 0

    #for keyword in user_keywords:
        #synonyms = [keyword] + synonym_dict.get(keyword, [])
        #match_count = 0
        #for syn in synonyms:
            #if syn in user_input:
                #match_count +=1
        #if match_count > 0:
            #covered_count += 1
            #total_match_count += match_count
        #else:
            #total_match_count += 0

    #base_score = covered_count

    #if covered_count == len(user_keywords):
        #bonus_score = (total_match_count - covered_count) * 0.1
    #else:
        #bonus_score = 0
    #return base_score + bonus_score
    
    for keyword in user_keywords:
        synonyms = [keyword] + synonym_dict.get(keyword, [])
        match_count = sum(syn in user_input for syn in synonyms)
        synonyms
        print(user_input)

        if match_count > 0:
            covered_count += 1
            total_match_count += match_count
            print(covered_count)
            print(total_match_count)
    
    return covered_count + total_match_count * 0.1



In [74]:
#マッチング処理の実施
df_unique2["hybrid_score"] = df_unique2["趣味詳細説明文"].astype(str).apply(
    lambda x: hybrid_keyword_score(user_input, user_keywords, synonym_dict)
)

# "hybrid_score" を降順に並べて、TOP3を抽出する
def get_top_n_hobby_name(df, column_name, n=3):
    return df.sort_values(by=column_name, ascending=False).head(n)

top_hobbies = get_top_n_hobby_name(df_unique2, "hybrid_score", n=3)

TOP1 = top_hobbies.iloc[0]["趣味名"]
TOP2 = top_hobbies.iloc[1]["趣味名"]
TOP3 = top_hobbies.iloc[2]["趣味名"]

print(TOP1)
print(TOP2)
print(TOP3)

あなたは趣味を通じて姿勢がよくなる、集中する、表現力を味わうことを望んでいます。またそれは一人で楽しむことができ、なおかつ自宅でできるものであると、より満足感が高まります。
あなたは趣味を通じて姿勢がよくなる、集中する、表現力を味わうことを望んでいます。またそれは一人で楽しむことができ、なおかつ自宅でできるものであると、より満足感が高まります。
1
1
あなたは趣味を通じて姿勢がよくなる、集中する、表現力を味わうことを望んでいます。またそれは一人で楽しむことができ、なおかつ自宅でできるものであると、より満足感が高まります。
あなたは趣味を通じて姿勢がよくなる、集中する、表現力を味わうことを望んでいます。またそれは一人で楽しむことができ、なおかつ自宅でできるものであると、より満足感が高まります。
あなたは趣味を通じて姿勢がよくなる、集中する、表現力を味わうことを望んでいます。またそれは一人で楽しむことができ、なおかつ自宅でできるものであると、より満足感が高まります。
1
1
あなたは趣味を通じて姿勢がよくなる、集中する、表現力を味わうことを望んでいます。またそれは一人で楽しむことができ、なおかつ自宅でできるものであると、より満足感が高まります。
あなたは趣味を通じて姿勢がよくなる、集中する、表現力を味わうことを望んでいます。またそれは一人で楽しむことができ、なおかつ自宅でできるものであると、より満足感が高まります。
あなたは趣味を通じて姿勢がよくなる、集中する、表現力を味わうことを望んでいます。またそれは一人で楽しむことができ、なおかつ自宅でできるものであると、より満足感が高まります。
1
1
あなたは趣味を通じて姿勢がよくなる、集中する、表現力を味わうことを望んでいます。またそれは一人で楽しむことができ、なおかつ自宅でできるものであると、より満足感が高まります。
あなたは趣味を通じて姿勢がよくなる、集中する、表現力を味わうことを望んでいます。またそれは一人で楽しむことができ、なおかつ自宅でできるものであると、より満足感が高まります。
あなたは趣味を通じて姿勢がよくなる、集中する、表現力を味わうことを望んでいます。またそれは一人で楽しむことができ、なおかつ自宅でできるものであると、より満足感が高まります。
1
1
あなたは趣味を通じて姿勢がよくなる、集中する、表現力を

In [70]:
top_hobbies

,カテゴリ,趣味名,趣味詳細説明文,hybrid_score
0,スポーツ,ライブ観戦,"ライブは""鑑賞する""、""聴く""というよりも、音楽を楽しみつつ、その場の臨場感も楽しむことから...",1.1
226,音楽,電子ピアノ,ピアノを始めてみたい！子供にピアノを習わせたい！\nしかし、ピアノは高価で家計に負担がかかっ...,1.1
234,音楽,ジャズサックス,サックスと聞けば、まずどんな音楽が思い浮かびますか？\n超有名なところでは名探偵コナンのメイ...,1.1


In [35]:
import sqlite3

conn = sqlite3.connect("hobby_data_cleaned.db")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()
print(tables)

[('hobbies',)]
